In [2]:
import pandas as pd

# Muat Datasets
train_path = "D:/File-File Kompetisi/Satria Data/data_training.csv"
test_path = "D:/File-File Kompetisi/Satria Data/data_testing_no_label.csv"
submission_path = "D:/File-File Kompetisi/Satria Data/Sample_Submission.csv"

# Baca data training 
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_submission = pd.read_csv(submission_path)

# Display informasi umum tentang data training
df_train.info(), df_train.head()

# Muat ulang datasets dengan delimiter yang sesuai
df_train = pd.read_csv(train_path, delimiter=';')
df_test = pd.read_csv(test_path, delimiter=';')

# Display informasi umum tentang data training
df_train.info(), df_train.head()

# Cek distribusi kelas target
df_train["Kategori_Gizi"].value_counts(normalize=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                 Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                 --------------  ----- 
 0   ID_Siswa;Usia;Berat_Badan;Tinggi_Badan;IMT;Asupan_Kalori;Aktivitas_Fisik;Kategori_Gizi;Tingkat_Kesulitan;Jenis_Kelamin;Frekuensi_Makan;Konsumsi_Sayur_Buah;Durasi_Tidur (Menit);Fast_Food_Per_Minggu;Riwayat_Penyakit  8000 non-null   object
dtypes: object(1)
memory usage: 62.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 15 columns):
 #   Column      

Kategori_Gizi
Gizi Lebih     0.451875
Gizi Kurang    0.341500
Gizi Baik      0.206625
Name: proportion, dtype: float64

Kesimpulan 
Gizi Lebih : 45.18 %
Gizi Kurang : 34.15 %
Gizi Baik : 20.66 % (Paling Sedikit) 

In [3]:
from sklearn.preprocessing import LabelEncoder

# Copy dataset untuk preprocessing
df_preprocessed = df_train.copy()

# Encoding variabel kategorikal
categorical_cols = ["Kategori_Gizi", "Tingkat_Kesulitan", "Jenis_Kelamin", "Riwayat_Penyakit"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_preprocessed[col] = le.fit_transform(df_preprocessed[col])
    label_encoders[col] = le

# Tampilkan hasil encoding untuk kategori gizi
label_encoders["Kategori_Gizi"].classes_, df_preprocessed["Kategori_Gizi"].unique()


(array(['Gizi Baik', 'Gizi Kurang', 'Gizi Lebih'], dtype=object),
 array([1, 0, 2]))

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

# Pisahkan fitur dan label
X = df_preprocessed.drop(columns=["ID_Siswa", "Kategori_Gizi"])
y = df_preprocessed["Kategori_Gizi"]

# Hitung class weights sebagai kebalikan dari proporsi kelas
class_weights = {i: 1/v for i, v in zip(y.unique(), y.value_counts(normalize=True))}
sample_weights = np.array([class_weights[label] for label in y])

# Bagi data menjadi train dan validation set
X_train, X_val, y_train, y_val, sw_train, sw_val = train_test_split(
    X, y, sample_weights, test_size=0.2, stratify=y, random_state=42
)

# Tampilkan class weights
class_weights


{np.int64(1): 2.213001383125864,
 np.int64(0): 2.928257686676427,
 np.int64(2): 4.839685420447671}

In [6]:
import xgboost as xgb
from sklearn.metrics import classification_report

# Inisialisasi model XGBoost dengan parameter dasar
model = xgb.XGBClassifier(
    objective="multi:softmax",
    num_class=3,
    eval_metric="mlogloss",
    scale_pos_weight=[class_weights[0], class_weights[1], class_weights[2]],
    random_state=42
)

# Latih model dengan sample weights
model.fit(X_train, y_train, sample_weight=sw_train)

# Prediksi pada validation set
y_pred = model.predict(X_val)

# Evaluasi model
report = classification_report(y_val, y_pred, target_names=label_encoders["Kategori_Gizi"].classes_)
report


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\xgboost\core.py:158: UserWarning: [20:54:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


'              precision    recall  f1-score   support\n\n   Gizi Baik       0.99      1.00      0.99       331\n Gizi Kurang       1.00      1.00      1.00       546\n  Gizi Lebih       1.00      1.00      1.00       723\n\n    accuracy                           1.00      1600\n   macro avg       1.00      1.00      1.00      1600\nweighted avg       1.00      1.00      1.00      1600\n'